# Transport-diffusion problems

Let $\Omega = (0,1) \times (0,1)$, $\mu> 0$, and $\boldsymbol{b}\in\mathbb{R}^2$. Assume sufficient regularity for the load function and Dirichlet boundary data (e.g. $f\in L^2(\Omega)$ and $g\in H^{\frac12}(\partial\Omega)$) and consider the differential problem:
\begin{equation*}
  \begin{cases}
    - \mu \Delta u + \boldsymbol{b} \cdot\boldsymbol{\nabla} u = f & \mbox{ in } \Omega, \\[5mm]
    u = g & \mbox{ on } \partial\Omega.
  \end{cases}
\end{equation*}

## Weak formulation

Let $\tilde{u}= u - u_g$, with $u_g\in H^1(\Omega)$ such that $(u_g)_{|\partial\Omega}=g$. Thus, we have $\tilde{u}_{|\partial\Omega}=0$.
We multiply by a test function $v\in H^1_0(\Omega)$ and integrate by parts over the domain $\Omega$ to obtain
$$
  \int_\Omega \mu \boldsymbol{\nabla}u \cdot \boldsymbol{\nabla} v \, d\boldsymbol{x} 
  + \int_\Omega (\boldsymbol{b} \cdot\boldsymbol{\nabla} u) v \, d\boldsymbol{x}
  = \int_\Omega f v \, d\boldsymbol{x},
$$
where, as a result of the choice of the functional space for the test $v$, the boundary contribution has vanished.

Defining the bilinear form $a:H^1_0(\Omega)\times H^1_0(\Omega)\to\mathbb{R}$ such that
$ a(u,v) :=  \int_\Omega \mu \boldsymbol{\nabla}u \cdot \boldsymbol{\nabla} v \, d\boldsymbol{x}+ \int_\Omega (\boldsymbol{b} \cdot\boldsymbol{\nabla} u) v \, d\boldsymbol{x}$ and replacing $u$ with $\tilde{u}$ in the left-hand side of the previous relation, leads to 
$$
  a(\tilde{u},v) = a(u,v) - a(u_g,v)
  = \int_\Omega f v \, d\boldsymbol{x}- a(u_g,v).
$$
Renaming, for the sake of simplicity, $\tilde{u}$ with $u$, the weak formulation of the transport diffusion problem reads:
$$
  \text{find } u \in H^1_0(\Omega) \text{ verifying }
  a(u,v) =
  \int_\Omega f v \, d\boldsymbol{x} - a(u_g,v), \qquad
  \forall v \in H^1_0(\Omega).
$$

### Skew-Symmetric form of the transport term

Observing that $\boldsymbol{b} \cdot\boldsymbol{\nabla} u = \boldsymbol{\nabla}\cdot(\boldsymbol{b}\ u)$, we notice that the transport contribution can be rewritten as
$$
\begin{aligned}
\int_\Omega (\boldsymbol{b} \cdot\boldsymbol{\nabla} u) v \, d\boldsymbol{x} &=
\frac12 \int_\Omega (\boldsymbol{b} \cdot\boldsymbol{\nabla} u) v \, d\boldsymbol{x} + 
\frac12 \int_\Omega \boldsymbol{\nabla}\cdot(\boldsymbol{b}\ u) v \, d\boldsymbol{x}
\\
&=
\frac12 \int_\Omega (\boldsymbol{b} \cdot\boldsymbol{\nabla} u) v \, d\boldsymbol{x} - 
\frac12 \int_\Omega u(\boldsymbol{b} \cdot\boldsymbol{\nabla} v) \, d\boldsymbol{x},
\end{aligned}
$$
where, to pass to the second line, we have integrated by parts. This skew-symmetric formulation of the transport operator, is particularly useful to infer the coercivity of $a$. Indeed, we have
$$
a(u,u) = \mu || \boldsymbol{\nabla}u ||_{L^2(\Omega)^2}^2 > 0.
$$

## Construction of the FE approximation with FEniCS

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
    
from fenics import *

We declare the problem data corresponding to the analytical solution given by
    \begin{equation*}
      u(x,y) = \frac{e^{-\mu^{-1}x}-1}{e^{-\mu^{-1}}-1}
             + \frac{e^{-\mu^{-1}y}-1}{e^{-\mu^{-1}}-1}.
    \end{equation*}
Since $\boldsymbol{\nabla}u = \frac1\mu
\left(\frac{-e^{-\mu^{-1}x}}{e^{-\mu^{-1}}-1},
\frac{-e^{-\mu^{-1}y}}{e^{-\mu^{-1}}-1} \right)$ we get
$$
- \mu \Delta u + \boldsymbol{b} \cdot\boldsymbol{\nabla} u = \frac\mu{\mu^2}\left(\frac{e^{-\mu^{-1}x}}{e^{-\mu^{-1}}-1}+\frac{e^{-\mu^{-1}y}}{e^{-\mu^{-1}}-1}\right) +
\frac1\mu\left(\frac{e^{-\mu^{-1}x}}{e^{-\mu^{-1}}-1}-\frac{e^{-\mu^{-1}y}}{e^{-\mu^{-1}}-1}\right) =0.
$$

In [ ]:
degree = 1

#mu = Constant(1)
mu = Constant(1e-2)
bx, by = 1.0, 1.0
b = Constant((bx, by))

f = Constant(0)
u_exact = Expression('(exp(bx * (x[0] - 1) / mu) - exp(-bx / mu)) / (1 - exp(-bx / mu)) + '
                     '(exp(by * (x[1] - 1) / mu) - exp(-by / mu)) / (1 - exp(-by / mu))',
                     mu=mu, bx=bx, by=by, degree=2*degree+1)

### Streamline diffusion stabilization

We implement the streamline diffusion method, assuming that the stabilization parameter $\tau$ is given by
$$
\tau = \frac{\delta h}{2||\boldsymbol{b}||}. 
$$
The case $\delta=0$ corresponds to the usual FEM without stabilization.

In [ ]:
def solve_stream_diff(n, degree, f, u_exact, delta):
  # 1. mesh generation
  mesh = UnitSquareMesh(n, n, 'crossed')

  # 2. definition of finite element space
  V = FunctionSpace(mesh, 'CG', degree)

  g_boundary = u_exact

  def boundary(x, on_boundary):
      return on_boundary

  bc = DirichletBC(V, g_boundary, boundary)

  # 3. assembling matrices and vectors/
  u = TrialFunction(V)
  v = TestFunction(V)

  h = CellDiameter(mesh)
  bnorm = sqrt(dot(b, b))
  tau_K = 0.5*delta*h/bnorm

  a = (mu * dot(grad(u), grad(v)) + dot(b, grad(u)) * v) * dx
  L = f * v * dx

  # Streamline diffusion stabilization
  a_stabilization = tau_K * dot(b, grad(u)) * dot(b, grad(v)) * dx 
  a = a + a_stabilization

  # 4. solving discrete problem
  u = Function(V)
  u.rename('solution', 'solution')
  solve(a == L, u, bc)
  
  return mesh, u

In [ ]:
# user dependent stabilization parameter
delta = Constant(1.0)

for n in [10, 20, 40, 80, 160]:
  mesh, u = solve_stream_diff(n, degree, f, u_exact, delta)
  print('n = {}, error L2 = {:.3e}, error H1 = {}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))

n = 10, error L2 = 1.833e-01, error H1 = 8.214359207655594
n = 20, error L2 = 1.056e-01, error H1 = 7.064457537984679
n = 40, error L2 = 5.625e-02, error H1 = 5.207001008520208
n = 80, error L2 = 2.899e-02, error H1 = 3.289735276932242
n = 160, error L2 = 1.484e-02, error H1 = 1.8685513753194471


### SUPG, GLS, and DW stabilization 

As stabilization parameter $\tau$ we take
$$
\tau = \frac{\delta h}{2||\boldsymbol{b}||} \min\left\{1,\frac{\mathcal{Pe}}3\right\}, \quad\text{with }\;\;  \mathcal{Pe}= \frac{||\boldsymbol{b}||h}{2\mu}. 
$$

In [ ]:
def solve_strong_cons(n, degree, f, u_exact, delta, rho):
  # 1. mesh generation
  mesh = UnitSquareMesh(n, n, 'crossed')

  # 2. definition of finite element space
  V = FunctionSpace(mesh, 'CG', degree)

  g_boundary = u_exact

  def boundary(x, on_boundary):
      return on_boundary

  bc = DirichletBC(V, g_boundary, boundary)

  # 3. assembling matrices and vectors/
  u = TrialFunction(V)
  v = TestFunction(V)

  h = CellDiameter(mesh)
  bnorm = sqrt(dot(b, b))
  tau_K = 0.5 * delta * h / conditional(bnorm * h > 6 * mu, bnorm, 6 * mu / h)

  a = (mu * dot(grad(u), grad(v)) + dot(b, grad(u)) * v) * dx
  L = f * v * dx

  A = lambda u: -mu * div(grad(u)) + dot(b, grad(u))
  A_S = lambda u: -mu * div(grad(u)) - 0.5 * div(b) * u
  A_SS = lambda u: dot(b, grad(u)) + 0.5 * div(b) * u
  # A_SS = lambda u: 0.5 * dot(b, grad(u)) + 0.5 * div(b * u)

  a_stabilization = tau_K * A(u) * (A_SS(v) + rho * A_S(v)) * dx
  
  a = a + a_stabilization

  # 4. solving discrete problem
  u = Function(V)
  u.rename('solution', 'solution')
  solve(a == L, u, bc)
  
  return mesh, u

In [ ]:
# SUPG (rho = 0), GLS (rho = 1), DW (rho = -1)
rho = Constant(0)

# user dependent stabilization parameter
delta = Constant(1.0)

for n in [10, 20, 40, 80, 160]:
  mesh, u = solve_strong_cons(n, degree, f, u_exact, delta, rho)
  print('n = {}, error L2 = {:.3e}, error H1 = {}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))

n = 10, error L2 = 1.833e-01, error H1 = 8.214359207655594
n = 20, error L2 = 1.056e-01, error H1 = 7.064457537984679
n = 40, error L2 = 3.987e-02, error H1 = 4.8405168176359314
n = 80, error L2 = 1.138e-02, error H1 = 2.7609100464985765
n = 160, error L2 = 2.959e-03, error H1 = 1.442220300286551


### Comparison for $\mathbb{P}^k$ FEM with $k\ge 2$

We have observed that, due to the absence of reaction terms and the use of $\mathbb{P}^1$ elements, the methods are equivalent. Using higher-order polynomial spaces we should be able to notice the differences.

In [ ]:
degree = 2

mu = Constant(1e-2)
bx, by = 1.0, 1.0
b = Constant((bx, by))

f = Constant(0)
u_exact = Expression('(exp(bx * (x[0] - 1) / mu) - exp(-bx / mu)) / (1 - exp(-bx / mu)) + '
                     '(exp(by * (x[1] - 1) / mu) - exp(-by / mu)) / (1 - exp(-by / mu))',
                     mu=mu, bx=bx, by=by, degree=2*degree+1)

for n in [4, 8, 16, 32, 64]:  
  # user dependent stabilization parameter
  delta = Constant(0.06)
  mesh, u = solve_stream_diff(n, degree, f, u_exact, delta)
  print('SDif: n = {}, error L2 = {:.3e}, error H1 = {:.3e}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))
  
  mesh, u = solve_strong_cons(n, degree, f, u_exact, delta, 1.0)
  print('GLS : n = {}, error L2 = {:.3e}, error H1 = {:.3e}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))
  
  mesh, u = solve_strong_cons(n, degree, f, u_exact, delta, 0.0)
  print('SUPG: n = {}, error L2 = {:.3e}, error H1 = {:.3e}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))
  
  mesh, u = solve_strong_cons(n, degree, f, u_exact, delta, -1.0)
  print('DW  : n = {}, error L2 = {:.3e}, error H1 = {:.3e}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))
  
  delta = Constant(0.0)
  mesh, u = solve_stream_diff(n, degree, f, u_exact, delta)
  print('NO_S: n = {}, error L2 = {:.3e}, error H1 = {:.3e}'.format(n, 
         errornorm(u_exact, u, 'L2'), errornorm(u_exact, u, 'H10')))
  print()

SDif: n = 4, error L2 = 1.178e-01, error H1 = 6.590e+00
GLS : n = 4, error L2 = 1.197e-01, error H1 = 6.640e+00
SUPG: n = 4, error L2 = 1.195e-01, error H1 = 6.608e+00
DW  : n = 4, error L2 = 1.194e-01, error H1 = 6.577e+00
NO_S: n = 4, error L2 = 1.664e-01, error H1 = 8.283e+00

SDif: n = 8, error L2 = 5.502e-02, error H1 = 5.435e+00
GLS : n = 8, error L2 = 5.549e-02, error H1 = 5.466e+00
SUPG: n = 8, error L2 = 5.597e-02, error H1 = 5.447e+00
DW  : n = 8, error L2 = 5.661e-02, error H1 = 5.435e+00
NO_S: n = 8, error L2 = 6.056e-02, error H1 = 6.001e+00

SDif: n = 16, error L2 = 2.019e-02, error H1 = 3.379e+00
GLS : n = 16, error L2 = 1.971e-02, error H1 = 3.386e+00
SUPG: n = 16, error L2 = 2.030e-02, error H1 = 3.378e+00
DW  : n = 16, error L2 = 2.116e-02, error H1 = 3.392e+00
NO_S: n = 16, error L2 = 2.035e-02, error H1 = 3.543e+00

SDif: n = 32, error L2 = 5.981e-03, error H1 = 1.547e+00
GLS : n = 32, error L2 = 4.795e-03, error H1 = 1.531e+00
SUPG: n = 32, error L2 = 5.045e-03, er